# WIP

In [ ]:
class AlphaSeqDataset(Dataset):
    """ AlphaSeq sequence dataset, embedded with ESM model. """

    def __init__(self, csv_file:str, device:str):
        self.df = pd.read_csv(csv_file, sep=',', header=0)
        self.device = device

        self.df = self.df[['POI', 'Sequence', 'Mean_Affinity']]
        self.df['variant'] = pd.qcut(self.df['Mean_Affinity'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4']) # Quantile based on mean affinity
        self.df = self.df.rename(columns={'POI':'seq_id', 'Sequence':'sequence'})

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx):
        return self.df.iloc[idx]['seq_id'], self.df.iloc[idx]['variant'], self.df.iloc[idx]['sequence']

In [ ]:
def plot_from_embedding(csv_file, type):
    df = pd.read_csv(csv_file, sep=',', header=0)

    # Custom color mapping for specified variants
    cmap = {'Q1': 'black', 
            'Q2': '#1f77b4',    # blue
            'Q3': '#d62728',    # red
            'Q4': '#f7b6d2'}    # pink

    variant_labels = sorted(df["variant"].unique())    
    variant_colors = {}
    for variant in variant_labels:
        if variant in cmap:
            variant_colors[variant] = cmap[variant]
            
    df['colors'] = [variant_colors[variant] for variant in df['variant']]

    # Legend handles
    legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=variant_colors[variant], label=variant) for variant in variant_labels]

    # Create a scatter plot
    plt.figure(figsize=(16,9))
    plt.rcParams['font.family'] = 'sans-serif'
    plt.scatter(df['DIM_1'], df['DIM_2'], c=df['colors'], s=20, edgecolor='w', alpha=0.7)
    plt.xlabel(f'{type} Dimension 1')
    plt.ylabel(f'{type} Dimension 2')
    plt.legend(handles=legend_handles, loc='upper right')
    plt.tight_layout()
    plt.savefig(csv_file.replace('_coordinates.csv', '_plot.pdf'), format='pdf')